# SMO implementation
Attempting to implement the SMO (https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/tr-98-14.pdf). 


First step, we wil need to be able to evaluate the SVM on
a single instance. 

I initially explored hmatrix, but then I realised that I don't need matrix inversion for this algorithm, so using Repa would be better
`--import Numeric.LinearAlgebra`


In [2]:
{- LANGUAGE XTypeOperators -}
import  Data.Array.Repa


In [3]:
type BaseVector = Array U DIM1 Double
type BaseScalar = Array U Z Double

type Sample = BaseVector
type Weights = BaseVector
type Threshold = BaseScalar
type Kernel = BaseVector -> BaseVector -> BaseScalar

data ClassLabel = Class1 | Class2 deriving (Show, Eq)


chooseClass :: Double -> ClassLabel
chooseClass res = if res >= 0 then Class1 else Class2
             
dot :: Kernel
dot a b = foldS (+) 0 ( a *^ b)              

svm :: Kernel -> Weights -> Threshold -> Sample -> ClassLabel
svm k w b x = let
                res = (w `k` x) -^ b
              in 
                chooseClass  $ res ! Z
                
linearSvm :: Weights -> Threshold -> Sample -> ClassLabel
linearSvm = svm dot

In [4]:
x_inputs = [1..10] :: [Double]
w_inputs = [11..20] :: [Double]
x = fromListUnboxed (Z :. (10::Int))  x_inputs
w = fromListUnboxed (Z :. (10::Int))  w_inputs

thresh = fromListUnboxed Z ([2] :: [Double])

svm dot w thresh x 

Class1

In [5]:
largethresh = fromListUnboxed Z ([1000] :: [Double])

svm dot w largethresh x 

Class2

# Build Helper Functions 
Now that we can evaluate the SVM we need to work towards building the simple functions described in the paper. 

In [7]:
-- equation 13
lowerAlpha :: Double -> Bool -> Double -> Double  -> Double
lowerAlpha c ueqY a1 a2 = let alphaDiff = (a1 - a2)
                          in 
                            case ueqY of
                                True -> max 0 (alphaDiff)
                                False -> max 0 (alphaDiff - c )

-- equation 14
upperAlpha :: Double -> Bool -> Double -> Double -> Double
upperAlpha c ueqY a1 a2 = let alphaDiff = (a1 - a2)
                          in
                            case ueqY of
                                True -> min c (c + alphaDiff)
                                False -> min c alphaDiff
                                

In [9]:
-- equation 15, 2nd Derivative
eta :: Kernel -> Sample -> Sample -> BaseScalar
eta k x1 x2 = (x1 `k` x1) +^ (x2 `k` x2) -^ 2 *^ (x1 `k` x2)


classToInt :: ClassLabel -> Integer
classToInt Class1 = 0
classToInt class2 = 1

classError :: ClassLabel -> ClassLabel -> Integer
classError label1 label2 = if label1 == label2 then 0 else 1


Couldn't match type `D' with `U'
Expected type: BaseScalar
  Actual type: Array D Z Double
In the expression: (x1 `k` x1) +^ (x2 `k` x2) -^ 2 *^ (x1 `k` x2)
In an equation for `eta': eta k x1 x2 = (x1 `k` x1) +^ (x2 `k` x2) -^ 2 *^ (x1 `k` x2)

In [6]:
-- WARNING I THINK I MAY HAVE MISINTERPRETED THE MEANING OF Y!!!

-- equation 16, minimum along contstraint direction
alpha2New :: BaseScalar      -- Current Alpha Value
             -> BaseScalar   -- Result of the graidient calculation
             -> ClassLabel   -- TrueLabel of point1
             -> ClassLabel   -- TrueLabel of point2
             -> ClassLabel   -- Predicted Label of point1
             -> ClassLabel   -- Predicted Label of point2 
             -> BaseScalar   -- New alpha
alpha2New a e y1 y2 s1 s2 = 
    let 
        y2' = classToInt y2
        e1 = classError y1 s1
        e2 = classError y2 s2
    in
        a + y2'* (e1-e2) / e -- Implementing eq 16
        
-- Equation 17, 
alphaNewClipped :: BaseScalar -- alpha2New 
                   -> Double  -- H
                   -> Double  -- L
                   -> BaseScalar
alphaNewClipped a h l 
    | a >= h = h -- will fail typechecker might need to cast or something...
    | a <= l = l
    | otherwise = a

-- Equation 18

alpha1New :: Double   -- Alpha1
             -> ClassLabel -- True Label class1
             -> ClassLabel -- True Label class2
             -> BaseScalar -- Alpha2
             -> BaseScalar -- Alpha2Clipped
alphaNew a y1 y2 a2 a2clip =
    let 
        y1' = classToInt y1
        y2' = classToInt y2
        s = y1'*y2' :: Double 
    in 
        a + s * (a2 - a2clip)

Parse error (line 20, column 50): parse error on input `$'
Perhaps you intended to use TemplateHaskell